
# 📊 Elasticity Project — Phase 2: Data Cleaning and Feature Engineering

---

## 📝 Purpose of this Notebook

This notebook initiates **Phase 2** of the elasticity modeling project:
- Clean the raw dataset after initial exploration
- Engineer features necessary for elasticity regression modeling
- Prepare a finalized dataset ready for modeling

---

## 📚 Tasks Covered

- Remove zero-sales observations to avoid skewing elasticity
- Create log-transformed sales feature (`Log_Sales`)
- Engineer promotional flags and seasonal features (Month, Weekday, Year)
- Output a clean dataset for modeling

---

## 🔥 Next Steps After This Notebook

- Model log-sales as a function of price and promotions
- Estimate price elasticity across stores and products
- Build a Streamlit dashboard to visualize elasticity curves

---

## 🚀 Let's Get Started!